# EJERCICIO 1

### El propósito de este ejercicio es identificar endogeneidad en nuestro modelo, para ello utilizaremos la prueba de Hausman.


#### 1.- Revisamos la correlación entre  la variable *"índice de protección contra la expropiación ($avexpr_i$)" y los errores ($u_i$)*

Esta prueba tiene el objetivo de identificar la existencia de endogenidad. Para ello proponemos la siguiente estructura de pruebas de hipótesis:
\begin{eqnarray*}
    H_0 &:& Cov(avexpr_i, u_i) = 0 \text{ no hay endogeneidad} \\
    H_1 &:& Cov(avexpr_i, u_i) \neq 0 \text{ endogeneidad}
\end{eqnarray*}

En esta tarea deberá implemantar la prueba.

La prueba de Hausman se corre en dos etapas:


In [1]:
#importamos las librerías necesarias
#!pip install linearmodels==1.0
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

#
import warnings
warnings.filterwarnings('ignore')

##### Para realizar este ejercicio utilice el conjunto de datos 'maketable4.dta'

In [2]:
# Impoprtación de los datos:
df4 = pd.read_stata('maketable4.dta')
df4.head()

,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco
0,AFG,0.000000,1.0,NaN,NaN,1.0,2.0,1.0,1.0,93.699997,4.540098,NaN,NaN
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0
4,ARM,0.000000,0.0,NaN,7.682482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 ##### Añade la constante

In [3]:
df4['constante'] = 1
df4.head()

,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco,constante
0,AFG,0.000000,1.0,NaN,NaN,1.0,2.0,1.0,1.0,93.699997,4.540098,NaN,NaN,1
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0,1
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0,1
4,ARM,0.000000,0.0,NaN,7.682482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


#  1a etapa:

Corremos una regresión entre *índice de protección contra la expropiación ($avexpr$) y el instrumento tasa de mortalidad de los colonizadores ($logem4$)

In [4]:
# Defina la vaeriable Endogena:
Endogena = df4['logpgp95']

# defina las variables Exogenas (+ una constante)
Exogena = df4[['avexpr', 'logem4', 'constante']]

# Ejecute la regresión
reg1 = sm.OLS(endog = Endogena,
              exog = Exogena,
              missing='drop').fit()

# 2a etapa:

Recuperamos los residuales  $ \hat{u}_i $

In [5]:
# Creamos una nueva variable con residuales
df4['resid'] = reg1.resid
df4.head()

,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco,constante,resid
0,AFG,0.000000,1.0,NaN,NaN,1.0,2.0,1.0,1.0,93.699997,4.540098,NaN,NaN,1,NaN
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0,1,0.442047
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0,1,0.986394
4,ARM,0.000000,0.0,NaN,7.682482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


####  Incluímos los residuales anteriores en la regresión original que relaciona: *'logpgp95: lgaritmo del pib per cápita en 1995 y avexpr:índice de protección contra la expropiación
$$
logpgp95_i = \beta_0 + \beta_1 avexpr_i + \alpha \hat{u}_i + u_i
$$

In [6]:
# Defina variables X y Y:

X = df4[['constante', 'avexpr', 'resid']]

Y = df4['logpgp95']

# Ejecutemos regresión
reg2 = sm.OLS(endog = Y,
              exog = X,
              missing='drop').fit()

#Imprimimos resultados
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     252.8
Date:                Sun, 30 Mar 2025   Prob (F-statistic):           6.11e-32
Time:                        23:42:40   Log-Likelihood:                -27.764
No. Observations:                  70   AIC:                             61.53
Df Residuals:                      67   BIC:                             68.27
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constante      4.7135      0.195     24.229      0.0

# Dado el valor p-value de los residuales, ¿qué podemos decir sobre la endogeneidad del modelo dada  avexpr y logpgp95 ?

Discuta brevemente.

Con un R2 de 0.883 el modelo explica el 88,3% de la variabilidad en el logaritmo del PIB
Todos los coeficientes tienen un pvalue de 0.000, por lo que son altamente significativos. Como resid es significativo en la segunda regresión el modelo sugiere que hay endogeneidad.

# EJERCICIO 2

* El parámetro $\beta$ puede ser calculado con álgebra matricial y el paquete numpy

* La ecuacón lineal que queremos estimar es: $ y = X\beta + u $

* Para resolver la ecuación para el parámetro $ \beta $,   minimizamos la suma del cuadrado de los residuales. $\underset{\hat{\beta}}{\min} \hat{u}'\hat{u}$

* Rearreglando la primera ecuación y substituyendo en la segunda ecuación podemo escribir:$\underset{\hat{\beta}}{\min} \ (Y - X\hat{\beta})' (Y - X\hat{\beta})$

* Resolviendo este problema de optimización obtenemos la solución para los coeficientes $ \hat{\beta} $
$$\hat{\beta} = (X'X)^{-1}X'y
$$
* Utilizando la información ya descrita, calcule  $ \hat{\beta} $ del modelo 1 utilizando numpy -
Sus resultados deberían ser los mismos que los resultados obtenidos en la clase de la semana pasada al correr:

   **X1 = ['const', 'avexpr']**

   **reg1 = sm.OLS(df2['logpgp95'], df2[X1], missing='drop').fit()**}
   
##### *Para  realizar este ejercicio utiliza los datos **'maketable1.dta'**

In [8]:
# Importar datos
df1 = pd.read_stata('maketable1.dta')

# Borre los NA's
df1 = df1.dropna(subset = ['logpgp95', 'avexpr'])
df1

,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0
5,AUS,98.000000,1.0,9.318182,9.897972,7.0,7.0,10.0,7.0,8.550000,2.145931,-0.170788,1.0
6,AUT,100.000000,0.0,9.727273,9.974877,NaN,NaN,NaN,NaN,NaN,NaN,-0.343900,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,YEM,0.000000,1.0,6.363636,6.646390,NaN,NaN,NaN,NaN,NaN,NaN,-1.551169,NaN
159,ZAF,22.000000,1.0,6.863636,8.885994,3.0,7.0,3.0,3.0,15.500000,2.740840,-1.386294,1.0
160,ZAR,8.000000,1.0,3.500000,6.866933,1.0,1.0,0.0,1.0,240.000000,5.480639,-3.411248,1.0
161,ZMB,3.000000,1.0,6.636364,6.813445,3.0,1.0,0.0,1.0,NaN,NaN,-2.975930,NaN


###### Añade la constante

In [9]:
df1['const'] = 1
df1.head()

,shortnam,euro1900,excolony,avexpr,logpgp95,cons1,cons90,democ00a,cons00a,extmort4,logem4,loghjypl,baseco,const
1,AGO,8.000000,1.0,5.363636,7.770645,3.0,3.0,0.0,1.0,280.000000,5.634789,-3.411248,1.0,1
2,ARE,0.000000,1.0,7.181818,9.804219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,ARG,60.000004,1.0,6.386364,9.133459,1.0,6.0,3.0,3.0,68.900002,4.232656,-0.872274,1.0,1
5,AUS,98.000000,1.0,9.318182,9.897972,7.0,7.0,10.0,7.0,8.550000,2.145931,-0.170788,1.0,1
6,AUT,100.000000,0.0,9.727273,9.974877,NaN,NaN,NaN,NaN,NaN,NaN,-0.343900,NaN,1


##### Define las variables X y Y

In [10]:
#
y = df1['logpgp95']

X = df1[['avexpr', 'const']]

#### Calcula Beta

In [11]:
#β_hat = np.linalg.solve(X.T @ X, X.T @ y)

β_hat = np.dot( np.linalg.inv( np.dot(X.T,X) ), np.dot(X.T, y) )

#### Imprime los resultados, ¿son similares sus resultados?



In [12]:
print(f'β_0 = {β_hat[0]:.2}')
print(f'β_1 = {β_hat[1]:.2}')

β_0 = 0.53
β_1 = 4.6
